In [193]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment
from openpyxl import load_workbook
import time
pd.options.mode.chained_assignment = None  # default='warn'

In [181]:
def combine(x):
    # Функция для группировки всех значений в строку разделенную ;
    return  ';'.join(x)

def check_unique(x):
    # Функция для нахождения разночтений в значениях
    temp_lst = x.split(';')
    temp_set = set(temp_lst)
    if 'nan' in temp_set:
        return 'Не заполнены значения площади лесотаксационного выдела!!!'
    else:
        return 'Все в порядке' if len(temp_set) == 1 else 'Ошибка!!!'

def main_check_unique(x):
    # Функция для проверки корректности заполнения площади выдела
    temp_str = ';'.join(x) # Склеиваем все значения
    temp_lst = temp_str.split(';') # Создаем список разбивая по ;
    temp_set = set(temp_lst) # Превращаем в множество
    if len(temp_set) > 1: # Если длина множества больше 1 то есть погрешности
        return 0
    elif'nan' in temp_set:# если есть нан то не заполнены площади выдела
        return 0
    else:# Если все в порядке то возвращаем единственный элемент списка
        return convert_to_float(temp_lst[0])

def convert_to_float(x):
    """
    Функция для конвертирования строки в флоат.
    :param x: строка
    :return:
    """
    # очищаем от пробелов и возможных заполнения ячейки пробелом
    
    try:
        return float(x)
    except ValueError:
        return 0
def convert_to_int_transfer(cell):
    """
    Функция для конвертации в int
    """
    try:
        return int(cell)
    except ValueError:
        return 0

In [191]:
group_rb_region_transfer_3_to_reestr = 0

file_transfer_to_upp = 'data/Сравнение таблиц и присвоение данных (площади)/Таблица 3 с дополнительными колонками..xlsx'
file_transfer_reestr = 'data/Сравнение таблиц и присвоение данных (площади)/2022-10-27_64_Реестр УПП с дополнительными колонками.xlsx'

# file_transfer_to_upp = 'data/Перенос из таблицы 3 в реестр УПП/Якутия октябрь_Таблица 3с доп. колонками.xlsx'
# file_transfer_reestr = 'data/Перенос из таблицы 3 в реестр УПП/Якутия 2022-11-27_72_Реестр УПП по новой форме.xlsx'

path_to_end_folder = 'data/'
header_table3 = 'data/Сравнение таблиц и присвоение данных (площади)/Заголовок Таблица 3 от 08.11.xlsx'

In [152]:
# Создаем список колонок которые нужно загрузить
use_cols_upp = list(range(6))
use_cols_table3 = list(range(34))

# Загружаем датафреймы
df = pd.read_excel(file_transfer_reestr,skiprows=8,usecols=use_cols_upp)# Реестр УПП
df_table_3 = pd.read_excel(file_transfer_to_upp,skiprows=6,usecols=use_cols_table3) # Таблица 3

In [153]:
df_table_3.head()

,1,2,3,4,5,6,7,8,9,10,...,25,26,27,28,29,30,31,32,33,34
0,2022-06-29,NaN,0.2,Республика Бурятия,Ангоянское,Ангоянское,NaN,32,1,13.0,...,0,0,0,0,0,1,201,NaN,0.0,2015
1,2022-06-29,NaN,0.2,Республика Бурятия,Ангоянское,Ангоянское,NaN,32,1,300.0,...,0,0,0,0,0,1,201,NaN,0.0,2020
2,2022-06-29,NaN,0.3,Республика Бурятия,Ангоянское,Ангоянское,NaN,32,1,500.0,...,0,0,0,0,0,1,201,NaN,0.0,2021
3,2022-06-29,NaN,2.3,Республика Бурятия,Ангоянское,Ангоянское,NaN,32,1,400.0,...,более 40%,более 40%,2022-04-05 00:00:00,1,0,1,103,NaN,2.3,2015
4,2022-06-29,NaN,0.8,Республика Бурятия,Улан-Удэнское,Березовское,NaN,39,8,2.0,...,более 40%,более 40%,2022-04-05 00:00:00,1,0,1,103,NaN,0.8,2015


In [154]:
df.head()

,1,2,3,4,5,6
0,Ангоянское,Ангоянское,NaN,32,1,13
1,Ангоянское,Ангоянское,NaN,32,2,7
2,Ангоянское,Ангоянское,NaN,32,3,11
3,Ангоянское,Ангоянское,NaN,32,5,10
4,Ангоянское,Ангоянское,NaN,32,7,78


In [155]:
# Приводим названия колонок к строковому виду, чтобы избежать возможных проблем с названиями колонок
df.columns = list(map(str,list(df.columns)))
# на всякий случай очищаем от пробельных символов
df.columns = list(map(lambda x:x.replace(" ",""),list(df.columns)))

In [156]:
"""
Соответствие названий колонок используемым в программе номерам колонок
Лесничество -1
Участковое лесничество- 2
Урочище - 3
Номер лесного квартала -4
Номер лесотаксационного выдела -5
Площадь лесотаксационного выдела, га - 6
"""

'\nСоответствие названий колонок используемым в программе номерам колонок\nЛесничество -1\nУчастковое лесничество- 2\nУрочище - 3\nНомер лесного квартала -4\nНомер лесотаксационного выдела -5\nПлощадь лесотаксационного выдела, га - 6\n'

In [157]:
df.rename(columns={'1':'Лесничество','2':'Участковое лесничество','3':'Урочище','4':'Номер лесного квартала','5':'Номер лесотаксационного выдела',
                  '6':'Площадь лесотаксационного выдела, га'},inplace=True)



In [158]:
# Заполняем Nan чтобы группировка прошла успешно
df['Урочище'] = df['Урочище'].fillna('Название урочища не заполнено')

In [159]:
df['Площадь лесотаксационного выдела, га'] = df['Площадь лесотаксационного выдела, га'].astype(str)

df['Площадь лесотаксационного выдела, га'] = df['Площадь лесотаксационного выдела, га'].apply(
    lambda x: x.replace(',', '.'))

# Группируем
df_upp = df.groupby(['Лесничество', 'Участковое лесничество', 'Урочище', 'Номер лесного квартала',
                       'Номер лесотаксационного выдела']).agg({'Площадь лесотаксационного выдела, га': main_check_unique,
                                                               })

df_upp=df_upp.astype(str)


In [160]:
# Извлекаем индексы в колонки
df_upp = df_upp.reset_index()

In [161]:
df_upp.head()


,Лесничество,Участковое лесничество,Урочище,Номер лесного квартала,Номер лесотаксационного выдела,"Площадь лесотаксационного выдела, га"
0,Ангоянское,Ангоянское,Название урочища не заполнено,32,1,13.0
1,Ангоянское,Ангоянское,Название урочища не заполнено,32,2,7.0
2,Ангоянское,Ангоянское,Название урочища не заполнено,32,3,11.0
3,Ангоянское,Ангоянское,Название урочища не заполнено,32,5,10.0
4,Ангоянское,Ангоянское,Название урочища не заполнено,32,7,78.0


In [162]:
# Обрабатываем колонки таблицы 3
# Приводим названия колонок к строковому виду, чтобы избежать возможных проблем с названиями колонок
df_table_3.columns = list(map(str,list(df_table_3.columns)))

# Очищаем на всякий случай от пробельных символов
df_table_3.columns = list(map(lambda x:x.strip(),list(df_table_3.columns)))

In [163]:
# делаем строковыми первые 5 колонок в датафрейме Реестр УПП
df_upp[['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела']] = df_upp[['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела']].astype(str)
df_upp['ID'] = df_upp.loc[:,['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела'] ].sum(axis=1)
df_upp['ID'] = df_upp['ID'].apply(lambda x:x.replace(' ',''))
df_upp['ID'] = df_upp['ID'].apply(lambda x:x.replace('nan',''))

In [164]:
# Заменяем nan в урочище 
df_table_3['7'] = df_table_3['7'].fillna('Название урочища не заполнено')

In [165]:
# делаем строковыми колонки по которым будем соединять в датафрейме 3 таблицы
df_table_3[['5','6','7','8','9']] = df_table_3[['5','6','7','8','9']].astype(str)
df_table_3['ID'] = df_table_3.loc[:,['5','6','7','8','9'] ].sum(axis=1)
df_table_3['ID'] = df_table_3['ID'].apply(lambda x:x.replace(' ',''))
df_table_3['ID'] = df_table_3['ID'].apply(lambda x:x.replace('nan',''))

In [173]:
# Мержим по полю айди
merged_df = pd.merge(df_upp,df_table_3,how='outer',left_on='ID',right_on='ID',indicator=True)

In [174]:
merged_df['Площадь'] = np.where(merged_df['_merge'] == 'both',merged_df['Площадь лесотаксационного выдела, га'],merged_df['10'])

In [175]:
merged_df.head()

,Лесничество,Участковое лесничество,Урочище,Номер лесного квартала,Номер лесотаксационного выдела,"Площадь лесотаксационного выдела, га",ID,1,2,3,...,27,28,29,30,31,32,33,34,_merge,Площадь
0,Ангоянское,Ангоянское,Название урочища не заполнено,32,1,13.0,АнгоянскоеАнгоянскоеНазваниеурочищанезаполнено321,2022-06-29,NaN,0.2,...,0,0.0,0.0,1.0,201.0,NaN,0.0,2015.0,both,13.0
1,Ангоянское,Ангоянское,Название урочища не заполнено,32,1,13.0,АнгоянскоеАнгоянскоеНазваниеурочищанезаполнено321,2022-06-29,NaN,0.2,...,0,0.0,0.0,1.0,201.0,NaN,0.0,2020.0,both,13.0
2,Ангоянское,Ангоянское,Название урочища не заполнено,32,1,13.0,АнгоянскоеАнгоянскоеНазваниеурочищанезаполнено321,2022-06-29,NaN,0.3,...,0,0.0,0.0,1.0,201.0,NaN,0.0,2021.0,both,13.0
3,Ангоянское,Ангоянское,Название урочища не заполнено,32,1,13.0,АнгоянскоеАнгоянскоеНазваниеурочищанезаполнено321,2022-06-29,NaN,2.3,...,2022-04-05 00:00:00,1.0,0.0,1.0,103.0,NaN,2.3,2015.0,both,13.0
4,Ангоянское,Ангоянское,Название урочища не заполнено,32,2,7.0,АнгоянскоеАнгоянскоеНазваниеурочищанезаполнено322,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,NaN


In [197]:
out_table3_df = merged_df[merged_df['_merge'] != 'left_only']

In [198]:
# Приводим к инту колонки чтобы отсортировать их
out_table3_df['8'] = out_table3_df['8'].apply(convert_to_int_transfer)
out_table3_df['9'] = out_table3_df['9'].apply(convert_to_int_transfer)

In [199]:
out_table3_df.sort_values(['5','6','7','8','9'],inplace=True)

In [200]:
# Присваиваем колонке 10 значение колонки созданной колонки 10
out_table3_df['10'] = out_table3_df['Площадь']

In [201]:
out_table3_df.drop(columns=['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела','Площадь лесотаксационного выдела, га','ID','_merge','Площадь'],inplace=True)

In [202]:
# Получаем текущую дату
current_time = time.strftime('%H_%M_%S %d.%m.%Y')
# Сохраняем отчет
# Для того чтобы увеличить ширину колонок для удобства чтения используем openpyxl
wb = load_workbook(header_table3) # Создаем объект
# Записываем результаты
for row in dataframe_to_rows(out_table3_df,index=False,header=False):
    wb['Лист1'].append(row)

In [203]:
wb.save(f'{path_to_end_folder}/Таблица 3 с присвоенными площадями из реестра УППП {current_time}.xlsx')

In [22]:

file_filing_reestr = 'data/Сравнение таблиц и присвоение данных (площади)/2022-10-27_64_Реестр УПП с дополнительными колонками.xlsx'
file_filing_table3 = 'data/Сравнение таблиц и присвоение данных (площади)/Таблица 3 с дополнительными колонками..xlsx'
path_to_end_folder_filling_area = 'data/'



In [ ]:
"""
Создаем датафреймы
Создаем датафрейм реестра считывая первые 5 колонок и пропуская шапку в 8 строк
"""

first_df = pd.read_excel(file_filing_reestr,skiprows=8,usecols=[0,1,2,3,4,5]) # датафрейм реестра

In [23]:
second_df = pd.read_excel(file_filing_table3,skiprows=6) # датафрейм таблицы 3 в которой нужно заполнить площадь

In [24]:
first_df.head(1)

,1,2,3,4,5,6
0,Ангоянское,Ангоянское,NaN,32,1,13


In [25]:
second_df.head(1)

,1,2,3,4,5,6,7,8,9,10,...,32,33,34,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40
0,2022-06-29,NaN,0.2,Республика Бурятия,Ангоянское,Ангоянское,NaN,32,1,13.0,...,NaN,0.0,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
second_df.columns

Index([          '1',           '2',           '3',           '4',
                   5,             6,             7,             8,
                   9,            10,            11,            12,
                  13,            14,            15,            16,
                  17,            18,            19,            20,
                  21,            22,            23,            24,
                  25,            26,            27,            28,
                  29,            30,            31,            32,
                  33,            34, 'Unnamed: 34', 'Unnamed: 35',
       'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39',
       'Unnamed: 40'],
      dtype='object')

In [27]:
# Переименовываем колонки в датафрейме реестра
first_df.columns = ['Лесничество','Участковое лесничество','Урочище','Квартал','Выдел','Площадь выдела']

In [28]:
# Приводим названия колонок к строковому формату
second_df.columns = list(map(str,second_df.columns))

In [29]:
# Удаляем колонки включающие unnamed
second_df.drop([col for col in second_df.columns if 'Unnamed' in col],axis=1,inplace=True)

In [30]:
second_df.columns

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25',
       '26', '27', '28', '29', '30', '31', '32', '33', '34'],
      dtype='object')

In [31]:
# заполняем урочище
first_df['Урочище'] = first_df['Урочище'].fillna('Название урочища не заполнено')
second_df['7'] = second_df['7'].fillna('Название урочища не заполнено')

In [32]:
# Обрабатываем колонку площади
first_df['Площадь выдела'] = first_df['Площадь выдела'].astype(str)
first_df['Площадь выдела'] = first_df['Площадь выдела'].apply(lambda x: x.replace('nan','0'))
first_df['Площадь выдела'] = first_df['Площадь выдела'].apply(lambda x: x.replace(' ','0'))
first_df['Площадь выдела'] = first_df['Площадь выдела'].apply(lambda x: x.replace(',','.'))
first_df['Площадь выдела'] = first_df['Площадь выдела'].apply(lambda x:x.strip())

In [33]:
# Группируем чтобы получить все возможные значения площади для каждого выдела
group_first_df = first_df.groupby(['Лесничество','Участковое лесничество','Урочище','Квартал','Выдел']).agg({'Площадь выдела':combine})

In [34]:
group_first_df['Контроль площади'] = group_first_df['Площадь выдела'].apply(check_unique_filling_area)

In [35]:
out_df=group_first_df.reset_index() # Превращаем индексы в колонки 


In [36]:
first_df.columns

Index(['Лесничество', 'Участковое лесничество', 'Урочище', 'Квартал', 'Выдел',
       'Площадь выдела'],
      dtype='object')

In [37]:
second_df.columns

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25',
       '26', '27', '28', '29', '30', '31', '32', '33', '34'],
      dtype='object')

In [38]:
# Получаем список колонок которые нужно привести к строковому виду
out_df_lst_columns = ['Лесничество','Участковое лесничество','Урочище','Квартал','Выдел']
second_lst_columns = ['Лесничество','Участковое лесничество','Урочище','Квартал','Выдел']

# Конвертируем все колонки в строковый формат и очищаем от пробельных символов
# Конвертируем нужные нам колонки в str
convert_name_columns_to_str(out_df,first_lst_columns)
convert_name_columns_to_str(second_df,second_lst_columns)

NameError: name 'first_lst_columns' is not defined

In [ ]:
# Создаем в каждом датафрейме колонку с айди путем склеивания всех нужных колонок в одну строку
out_df['ID'] = out_df.loc[:,out_df_lst_columns].sum(axis=1)
second_df['ID'] = second_df.loc[:,second_lst_columns].sum(axis=1)

In [ ]:
out_df

In [ ]:
# Создаем документ
wb = openpyxl.Workbook()
# создаем листы
ren_sheet = wb['Sheet']
ren_sheet.title = 'Выделы только в реестре'
wb.create_sheet(title='Выделы только в таблице 3', index=1)
wb.create_sheet(title='Площадь для совпадающих выделов', index=2)

# Проводим слияние
itog_df = pd.merge(out_df, second_df, how='outer', left_on=['ID'], right_on=['ID'],
                   indicator=True)

# Записываем каждый датафрейм в соответсвующий лист
left_df = itog_df[itog_df['_merge'] == 'left_only']
left_df.drop(['_merge','ID','Лесничество_y','Участковое лесничество_y','Урочище_y','Квартал_y','Выдел_y','Площадь выдела_y'], axis=1, inplace=True)
left_df.columns = ['Лесничество','Участковое лесничество','Урочище','Квартал','Выдел','Значения площади выдела','Контроль площади']

for r in dataframe_to_rows(left_df, index=False, header=True):
    wb['Выделы только в реестре'].append(r)


In [ ]:
right_df = itog_df[itog_df['_merge'] == 'right_only']
right_df.drop(['_merge','ID','Лесничество_x','Участковое лесничество_x','Урочище_x','Квартал_x','Выдел_x','Площадь выдела_x','Площадь выдела_y','Контроль площади'], axis=1, inplace=True)
right_df.columns= ['Лесничество','Участковое лесничество','Урочище','Квартал','Выдел']
# Добавляем временную колонку чтобы по ней провести группировку
right_df['temp'] = '1'
# Группируем чтобы оставить только уникальные выделы


group_right_df = right_df.groupby(['Лесничество','Участковое лесничество','Урочище','Квартал','Выдел']).agg({'temp':combine})
# Извлекаем индексы
group_right_df = group_right_df.reset_index()
# Удаляем колонку по которой проводилась группировка
group_right_df.drop(['temp'],axis=1, inplace=True)


for r in dataframe_to_rows(group_right_df, index=False, header=True):
    wb['Выделы только в таблице 3'].append(r)

In [ ]:
both_df = itog_df[itog_df['_merge'] == 'both']
both_df['Площадь выдела']= both_df['Контроль площади']

both_df.drop(['_merge','Площадь выдела_x','ID','Лесничество_y','Участковое лесничество_y','Урочище_y','Квартал_y','Выдел_y','Контроль площади','Площадь выдела_y'], axis=1, inplace=True)
both_df.columns = ['Лесничество','Участковое лесничество','Урочище','Квартал','Выдел','Площадь выдела']

In [ ]:
both_df

In [ ]:
for r in dataframe_to_rows(both_df, index=False, header=True):
    wb['Площадь для совпадающих выделов'].append(r)

In [ ]:
# Сохраняем
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
# Сохраняем итоговый файл
wb.save(f'{path_to_end_folder_filling_area}/Площади для таблицы 3 от {current_time}.xlsx')
